In [6]:
import os
import json
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [7]:
class MSCOCODataset(Dataset):
    def __init__(self, image_dir, annotation_file, transform=None):

        self.image_dir = image_dir
        self.transform = transform
        
        with open(annotation_file, 'r') as f:
            coco_data = json.load(f)
        
        self.images = {img['id']: img['file_name'] for img in coco_data['images']}
        self.annotations = coco_data['annotations']
        
        self.image_captions = {}
        for annotation in self.annotations:
            image_id = annotation['image_id']
            caption = annotation['caption']
            if image_id not in self.image_captions:
                self.image_captions[image_id] = []
            self.image_captions[image_id].append(caption)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get image ID and file name
        image_id = list(self.images.keys())[idx]
        file_name = self.images[image_id]
        
        img_path = os.path.join(self.image_dir, file_name)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        captions = self.image_captions[image_id]
        
        return image, captions

In [8]:
class MSCOCODatasetImages(Dataset):
    def __init__(self, root, annFile, transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path)).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.ids)

In [9]:
class MSCOCODatasetCaptions(Dataset):
    def __init__(self, image_dir, annotation_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform

        with open(annotation_file, 'r') as f:
            coco_data = json.load(f)

        self.images = {img['id']: img['file_name'] for img in coco_data['images']}
        self.annotations = coco_data['annotations']

        self.image_captions = {}
        for annotation in self.annotations:
            image_id = annotation['image_id']
            caption = annotation['caption']
            if image_id not in self.image_captions:
                self.image_captions[image_id] = []
            self.image_captions[image_id].append(caption)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_id = list(self.images.keys())[idx]
        captions = self.image_captions[image_id]

        return captions